In [216]:
using JuMP
using BenchmarkTools

# Sparse / normal version

We'll create num_var variables each with A x B entries
Each variable will be anonymous and added to an array

Next we'll build num_expr expressions from random combinations of the variables
with fixed coefficients and constants. We could switch to random coefficients
and constants but that shouldn't make a massive difference to what we're measuring

We'll do all of this within a function so that we can benchmark the runtime and memory usage

In [217]:
function setup_model()
    return Model()
end

setup_model (generic function with 1 method)

In [218]:
# Create a set of variables with with the given dimensions and stride.
# Save them to an array and return the array.
function create_variables(m::Model, num_var::Int64, dims::Tuple{Int64, Int64}, stride::Int64)
    if stride == 1
        vars = Array{Matrix{VariableRef}, 1}(undef, num_var)
    else
        vars = Array{JuMP.Containers.DenseAxisArray{VariableRef, 2, Tuple{StepRange{Int64, Int64}, StepRange{Int64, Int64}}, Tuple{JuMP.Containers._AxisLookup{Dict{Int64, Int64}}, JuMP.Containers._AxisLookup{Dict{Int64, Int64}}}}, 1}(undef, num_var)
    end
    for i in 1:num_var
        vars[i] = @variable(m, [1:stride:dims[1], 1:stride:dims[2]])
    end
    return vars
end

# Determine which variables to combine in each expression, and with what coefficients and constants.
# We'll do this separately so the same variables can be used in both dense and sparse expressions.
function choose_variables(num_var::Int64, num_expr::Int64, max_length::Int64)
    # Choose how many variables to use in each expression
    num_terms = rand(2:max_length, num_expr)

    # Choose which variables to use in this expression
    expr_var_idxs = Array{Array{Int64, 1}, 1}(undef, num_expr)
    expr_coeffs = Array{Array{Float64, 1}, 1}(undef, num_expr)
    for i in 1:num_expr
        expr_var_idxs[i] = rand(1:num_var, num_terms[i])
        expr_coeffs[i] = rand(num_terms[i])
    end

    return expr_var_idxs, expr_coeffs
end

function fill_with_zeros!(expr::AbstractArray{JuMP.GenericAffExpr{Float64,VariableRef},C}) where C
    for i in eachindex(expr)
        expr[i] = AffExpr(0.0)
    end
end

# function construct_expr(temp_expr::AbstractArray{JuMP.GenericAffExpr{Float64,VariableRef},C}, expr_var_idxs::Array{Int64, 1}, expr_coeffs::Array{Float64, 1}, vars::Vector{Matrix{VariableRef}}) where C
#     for j in eachindex(expr_var_idxs)
#         busy_work += sum(rand(dims[1] * dims[2] ÷ stride ÷ stride) .* 3.0 .+ 1.0)
#         for k in eachindex(vars[expr_var_idxs[j]])
#             add_to_expression!(temp_expr[k], expr_coeffs[j], vars[expr_var_idxs[j]][k])
#         end
#     end
#     return temp_expr
# end

# function create_expressions(m::Model, vars::Array{Matrix{VariableRef}, 1}, expr_var_idxs::Array{Array{Int64, 1}, 1}, expr_coeffs::Array{Array{Float64, 1}, 1}, dims::Tuple{Int64, Int64}, stride::Int64)
#     exprs = Array{Array{JuMP.GenericAffExpr{Float64,VariableRef},2}, 1}(undef, length(expr_var_idxs))
#     for i in eachindex(expr_var_idxs)
#         temp = Array{JuMP.GenericAffExpr{Float64,VariableRef},2}(undef, dims[1], dims[2])
#         fill_with_zeros!(temp)
#         exprs[i] = construct_expr(temp, expr_var_idxs[i], expr_coeffs[i], vars)
#     end
#     return exprs
# end

function create_expressions(m::Model, vars::Array{Matrix{VariableRef}, 1}, expr_var_idxs::Array{Array{Int64, 1}, 1}, expr_coeffs::Array{Array{Float64, 1}, 1}, dims::Tuple{Int64, Int64}, stride::Int64)
    exprs = Array{Array{JuMP.GenericAffExpr{Float64,VariableRef},2}, 1}(undef, length(expr_var_idxs))
    busy_work = 0.0
    for i in eachindex(expr_var_idxs)
        temp = JuMP.Containers.@container([i=1:dims[1],j=1:dims[2]], AffExpr(0.0))
        for j in eachindex(expr_var_idxs[i])
            busy_work += sum(rand(dims[1] * dims[2] ÷ stride ÷ stride) .* 3.0 .+ 1.0)
            for k in eachindex(vars[expr_var_idxs[i][j]])
                add_to_expression!(temp[k], expr_coeffs[i][j], vars[expr_var_idxs[i][j]][k])
            end
        end
        exprs[i] = temp
    end
    return exprs, busy_work
end

function create_expressions(m::Model, vars::Array{JuMP.Containers.DenseAxisArray{VariableRef, 2, Tuple{StepRange{Int64, Int64}, StepRange{Int64, Int64}}, Tuple{JuMP.Containers._AxisLookup{Dict{Int64, Int64}}, JuMP.Containers._AxisLookup{Dict{Int64, Int64}}}}, 1}, expr_var_idxs::Array{Array{Int64, 1}, 1}, expr_coeffs::Array{Array{Float64, 1}, 1}, dims::Tuple{Int64, Int64}, stride::Int64)
    exprs = Array{AbstractArray{JuMP.GenericAffExpr{Float64,VariableRef},2}, 1}(undef, length(expr_var_idxs))
    busy_work = 0.0
    for i in eachindex(expr_var_idxs)
        temp = JuMP.Containers.@container([i=1:stride:dims[1],j=1:stride:dims[2]], AffExpr(0.0))
        for j in eachindex(expr_var_idxs[i])
            busy_work += sum(rand(dims[1] * dims[2] ÷ stride ÷ stride) .* 3.0 .+ 1.0)
            for k in eachindex(vars[expr_var_idxs[i][j]])
                add_to_expression!(temp[k], expr_coeffs[i][j], vars[expr_var_idxs[i][j]][k])
            end
        end
        exprs[i] = temp
    end
    return exprs, busy_work
end

function sparse_approach(m::Model, expr_var_idxs::Vector{Vector{Int64}}, expr_coeffs::Vector{Vector{Float64}}, num_var::Int64, num_expr::Int64, max_length::Int64, dims::Tuple{Int64, Int64}, stride::Int64)
    vars = create_variables(m, num_var, dims, stride);
    exprs, a = create_expressions(m, vars, expr_var_idxs, expr_coeffs, dims, stride);
    return vars, exprs, a
end

sparse_approach (generic function with 1 method)

In [219]:
num_var = 100
num_expr = 50
max_length = num_var

expr_var_idxs, expr_coeffs = choose_variables(num_var, num_expr, max_length);

# One version using 1:N indexing and arrays (arr)
dims_arr = (20, 20)
stride_arr = 1

# One version using 1:stride:N indexing and dense containers (dc)
# Try to make them the same number of elements
dims_dc = (80, 80)
stride_dc = 4
# 80 / 4 * 80 / 4 = 400 elements

4

In [220]:
# Benchmarking settings:
evals = 1
samples = 10000 
max_time_seconds = 60

60

Array version of sparse approach

In [221]:
sparse_array = @benchmark sparse_approach(m, $expr_var_idxs, $expr_coeffs, $num_var, $num_expr, $max_length, $dims_arr, $stride_arr) setup=(m = setup_model()) evals=evals samples=samples seconds=max_time_seconds;

DenseContainer version of sparse approach

In [222]:
sparse_densecontainer = @benchmark sparse_approach(m, $expr_var_idxs, $expr_coeffs, $num_var, $num_expr, $max_length, $dims_dc, $stride_dc) setup=(m = setup_model()) evals=evals samples=samples seconds=max_time_seconds;

# Dense approach

Here we'll imagine that all prep work to gather data and calculate coefficients and constants has been done, so all the variables and expressions can be added to the model in one go. 

To do so, we'll store them in an array, indexed by the variable or expression number. They can then be accessed via a map of model indices <--> storage array indices.

In [223]:
total_number_var = num_var * dims_arr[1] * dims_arr[2] / stride_arr / stride_arr
total_number_expr = num_expr * dims_arr[1] * dims_arr[2] / stride_arr / stride_arr

function dense_array_approach(m::Model, expr_var_idxs::Vector{Vector{Int64}}, expr_coeffs::Vector{Vector{Float64}}, num_var::Int64, num_expr::Int64, max_length::Int64, dims::Tuple{Int64, Int64}, stride::Int64)
    busy_work = 0.0
    for x in 1:num_var
        busy_work += sum(rand(dims[1] * dims[2] ÷ stride ÷ stride) .* 3.0 .+ 1.0)
    end
    vars = @variable(m, [1:(dims[1] ÷ stride), 1:(dims[2] ÷ stride), 1:num_var])
    exprs = @expression(m, 
        [i=1:(dims[1] ÷ stride), j=1:(dims[2] ÷ stride), k=1:num_expr],
        sum(expr_coeffs[k][l] * vars[i, j, expr_var_idxs[k][l]] for l in 1:length(expr_var_idxs[k]))
    )
    return vars, exprs, busy_work
end

dense_array_approach (generic function with 1 method)

In [224]:
dense_array = @benchmark dense_array_approach(m, $expr_var_idxs, $expr_coeffs, $num_var, $num_expr, $max_length, $dims_arr, $stride_arr) setup=(m = setup_model()) evals=evals samples=samples seconds=max_time_seconds;

In [225]:
dense_densecontainer = @benchmark dense_array_approach(m, $expr_var_idxs, $expr_coeffs, $num_var, $num_expr, $max_length, $dims_dc, $stride_dc) setup=(m = setup_model()) evals=evals samples=samples seconds=max_time_seconds;

# Comparisons

In [226]:
m1 = setup_model()
v1, e1 = sparse_approach(m1, expr_var_idxs, expr_coeffs, num_var, num_expr, max_length, dims_arr, stride_arr);

m2 = setup_model()
v2, e2 = sparse_approach(m2, expr_var_idxs, expr_coeffs, num_var, num_expr, max_length, dims_dc, stride_dc);

m3 = setup_model()
v3, e3 = dense_array_approach(m3, expr_var_idxs, expr_coeffs, num_var, num_expr, max_length, dims_arr, stride_arr);

m4 = setup_model()
v4, e4 = dense_array_approach(m4, expr_var_idxs, expr_coeffs, num_var, num_expr, max_length, dims_dc, stride_dc);

# We can visually compare these, but they won't be the same using `==` right now because they're different models

([_[1] _[21] … _[361] _[381]; _[2] _[22] … _[362] _[382]; … ; _[19] _[39] … _[379] _[399]; _[20] _[40] … _[380] _[400];;; _[401] _[421] … _[761] _[781]; _[402] _[422] … _[762] _[782]; … ; _[419] _[439] … _[779] _[799]; _[420] _[440] … _[780] _[800];;; _[801] _[821] … _[1161] _[1181]; _[802] _[822] … _[1162] _[1182]; … ; _[819] _[839] … _[1179] _[1199]; _[820] _[840] … _[1180] _[1200];;; … ;;; _[38801] _[38821] … _[39161] _[39181]; _[38802] _[38822] … _[39162] _[39182]; … ; _[38819] _[38839] … _[39179] _[39199]; _[38820] _[38840] … _[39180] _[39200];;; _[39201] _[39221] … _[39561] _[39581]; _[39202] _[39222] … _[39562] _[39582]; … ; _[39219] _[39239] … _[39579] _[39599]; _[39220] _[39240] … _[39580] _[39600];;; _[39601] _[39621] … _[39961] _[39981]; _[39602] _[39622] … _[39962] _[39982]; … ; _[39619] _[39639] … _[39979] _[39999]; _[39620] _[39640] … _[39980] _[40000]], [1.1776807539441563 _[27201] + 0.3999000953886849 _[22001] + 1.1657719131443565 _[13201] + 0.053840580748412026 _[6401]

Sparse approach with arrays

In [227]:
sparse_array

BenchmarkTools.Trial: 817 samples with 1 evaluation.
 Range (min … max):  60.940 ms … 93.259 ms  ┊ GC (min … max): 0.00% … 25.11%
 Time  (median):     72.099 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   73.429 ms ±  7.658 ms  ┊ GC (mean ± σ):  8.89% ±  8.30%

      ▃▁▃▆ ▃▂▁▅█▇▃▅▁▂▂        ▁▁▁▁▄ ▂▁                         
  ▂▂█▇████████████████▆▆█▅▃▇▆██████▆██▅▄█▆█▅▇█▇▆▄▆▆▆▇█▇▄▄▃▄▂▆ ▅
  60.9 ms         Histogram: frequency by time        89.2 ms <

 Memory estimate: 99.00 MiB, allocs estimate: 325146.

Dense approach with arrays

In [228]:
dense_array

BenchmarkTools.Trial: 866 samples with 1 evaluation.
 Range (min … max):  56.997 ms … 87.769 ms  ┊ GC (min … max): 0.00% … 18.80%
 Time  (median):     68.029 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   69.313 ms ±  7.499 ms  ┊ GC (mean ± σ):  8.97% ±  8.69%

    ▁▂  ▁   ▁▁      ▅▅▅▆█▃▂  ▁ ▃▂                              
  ▄███▇▇█▇█▇██▇▇▆▇▇▆████████▆█▇███▆▇▅▆▄▃▆▆▃▆▃▆▆▅▅▄▆▆▆▄▃▇▄▆▅▃▂ ▄
  57 ms           Histogram: frequency by time        85.2 ms <

 Memory estimate: 84.70 MiB, allocs estimate: 315853.

Sparse approach with dense containers

In [229]:
sparse_densecontainer

BenchmarkTools.Trial: 794 samples with 1 evaluation.
 Range (min … max):  61.894 ms … 98.803 ms  ┊ GC (min … max): 0.00% … 17.63%
 Time  (median):     74.048 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   75.529 ms ±  8.155 ms  ┊ GC (mean ± σ):  8.91% ±  8.35%

      ▂█▁▂▄▆▇▅ ▇▄▇▄▄▅▄ ▂▂     ▁▁▁▅▂▁  ▂▃ ▁          ▂          
  ▂▃▃▇███████████████████▅▅▅▅███████████▅█▇▇▇▇▇███▆▄█▆▇▇▅▄▄▃▃ ▅
  61.9 ms         Histogram: frequency by time        92.3 ms <

 Memory estimate: 98.63 MiB, allocs estimate: 327396.

Dense approach with dense containers

In [230]:
dense_densecontainer

BenchmarkTools.Trial: 861 samples with 1 evaluation.
 Range (min … max):  57.201 ms … 89.931 ms  ┊ GC (min … max): 0.00% … 23.98%
 Time  (median):     68.375 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   69.714 ms ±  7.669 ms  ┊ GC (mean ± σ):  9.07% ±  8.59%

    ▃▃▂ ▂  ▂ ▂    ▁▄▅▂▂██▁ ▂▁▃ ▃                               
  ▅▆███▇█▇██▇█▇█▆▇████████████▅██▇▅█▄█▅▅▆▅▆▅▆▇▅█▇▄▆▆▄▆▆▆▅▅▄▁▄ ▅
  57.2 ms         Histogram: frequency by time        86.4 ms <

 Memory estimate: 84.70 MiB, allocs estimate: 315853.